In [1102]:
#packages
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.linalg import qr
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from autograd import grad, hessian
import autograd.numpy as anp
from sklearn.metrics import precision_score, recall_score, f1_score

In [1103]:
def sketch_matrix(m, n_columns, non_zero_entries):
    """Generates a sketching matrix S with random ±1 entries."""
    S = np.zeros((m, n_columns))
    scaling_factor = 1 / np.sqrt(non_zero_entries)
    
    for col in range(n_columns):
        nz_positions = np.random.choice(m, non_zero_entries, replace=False)
        values = np.random.choice([1, -1], non_zero_entries) * scaling_factor
        for idx, value in zip(nz_positions, values):
            S[idx, col] = value
    
    return S

#f : function
def unconstrained_newton_sketch(f, x0, m, A, b, non_zero_entries,tolerance = 1e-4, a = 0.1, b_factor = 0.5 , max_iter=1000):
    grad_f = grad(f)  # Gradient of f
    hess_f = hessian(f)  # Hessian of f
    
    xt = x0  # Starting point
    n = len(x0)
    for t in range(max_iter):
        print(t)
        # Generate the sketching matrix at each iteration
        St = sketch_matrix(m, n, non_zero_entries)
        #print(f"Sketching matrix at iteration {t}:\n{St}")

        # Compute the gradient and Hessian at the current point using autograd
        grad_value = grad_f(xt, A, b)  # Gradient at xt
        hess_value = hess_f(xt, A, b)  # Hessian at xt
        #print(f"Iteration {t} - grad_value:\n{grad_value}")
        #print(f"Iteration {t} - hess_value:\n{hess_value}")

        #ensure it is invertible
        regularization_strength = 1e-3
        sketched_hessian = St @ hess_value @ St.T
        sketched_hessian += regularization_strength * np.eye(sketched_hessian.shape[0])
        sketched_grad = St @ grad_value
        delta_xt_sketched = np.linalg.solve(sketched_hessian, sketched_grad)
        delta_xt = St.T @ delta_xt_sketched

        # Compute the approximate Newton decrement
        lambda_t = np.dot(grad_value, delta_xt)
        print(f"Iteration {t} - xt update norm: {np.linalg.norm(delta_xt)}")
        # Check stopping condition
        if lambda_t**2 / 2 <= tolerance:
            break
        
        # Backtracking line search
        step_size = 1.0
        while f(xt - step_size * delta_xt, A, b) > f(xt, A, b) - a * step_size * lambda_t:
            step_size *= b_factor
        
        # Update
        xt = xt - step_size * delta_xt
    
    return xt, np.abs(lambda_t)

#function to be used
def least_squares_loss(x, A, b):
    """
    Computes the least squares loss: f(x) = ||Ax - b||_2^2
    """
    return anp.sum((A @ x - b) ** 2)

def logistic_loss(x, A, b):
    """
    Computes the logistic loss for binary classification
    """
    z = anp.dot(A, x)  # Use anp.dot instead of @
    return anp.sum(anp.logaddexp(0, -b * z))

In [1104]:
df = pd.read_excel("../Dataset/employability.xlsx")

In [1105]:
df

,Name of Student,GENERAL APPEARANCE,MANNER OF SPEAKING,PHYSICAL CONDITION,MENTAL ALERTNESS,SELF-CONFIDENCE,ABILITY TO PRESENT IDEAS,COMMUNICATION SKILLS,Student Performance Rating,CLASS
0,Student 1,4,5,4,5,5,5,5,5,Employable
1,Student 2,4,4,4,4,4,4,3,5,Employable
2,Student 3,4,3,3,3,3,3,2,5,LessEmployable
3,Student 4,3,3,3,2,3,3,3,5,LessEmployable
4,Student 5,4,4,3,3,4,4,3,5,Employable
...,...,...,...,...,...,...,...,...,...,...
2977,Student 2996,4,3,3,3,3,3,2,5,Employable
2978,Student 2997,3,4,4,4,4,4,4,5,Employable
2979,Student 2998,4,5,4,5,4,4,4,5,Employable
2980,Student 2999,4,4,4,3,4,4,3,5,LessEmployable


In [1106]:
# Drop non-numeric columns (like student names)
df = df.drop(columns=["Name of Student"])

df["CLASS"] = df["CLASS"].map({"Employable": 1, "LessEmployable": 0})

# Separate features (A) and target variable (b)
A = df.drop(columns=["CLASS"]).values  # Convert features to NumPy array
b = df["CLASS"].values  # Target variable
A = A.astype(np.float64)
b = b.astype(np.float64)

In [1107]:
A

array([[4., 5., 4., ..., 5., 5., 5.],
       [4., 4., 4., ..., 4., 3., 5.],
       [4., 3., 3., ..., 3., 2., 5.],
       ...,
       [4., 5., 4., ..., 4., 4., 5.],
       [4., 4., 4., ..., 4., 3., 5.],
       [4., 4., 4., ..., 4., 4., 5.]])

In [1108]:
b

array([1., 1., 0., ..., 1., 0., 1.])

In [1109]:
x0 = np.random.randn(A.shape[1]) #columns = number of features
m = 10  # Number of rows in sketch matrix
non_zero_entries = 10

In [1110]:
print(f"A shape: {A.shape}")
print(f"x shape: {x0.shape}")
print(f"b shape: {b.shape}")

A shape: (2982, 8)
x shape: (8,)
b shape: (2982,)


In [1111]:
optimized_x, final_lambda = unconstrained_newton_sketch(logistic_loss, x0, m, A, b, non_zero_entries)

0
Iteration 0 - xt update norm: 26848.272070814815
1
Iteration 1 - xt update norm: 1231.194653355935
2
Iteration 2 - xt update norm: 3.1995383139480182
3
Iteration 3 - xt update norm: 0.26417777369409
4
Iteration 4 - xt update norm: 0.2412968605391625
5
Iteration 5 - xt update norm: 0.18115587843089062
6
Iteration 6 - xt update norm: 0.13921996413179802
7
Iteration 7 - xt update norm: 0.14544765018977154
8
Iteration 8 - xt update norm: 0.12531582237798877
9
Iteration 9 - xt update norm: 0.15388232878078348


In [1112]:
y_pred = 1 / (1 + np.exp(-A @ optimized_x)) 
y_pred = (y_pred > 0.5).astype(int)

In [1113]:
precision = precision_score(b, y_pred)
recall = recall_score(b, y_pred)
f1 = f1_score(b, y_pred)
accuracy = np.mean(y_pred == b)


print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Model Accuracy: 61.80%
Precision: 0.6028591352859135
Recall: 1.0
F1 Score: 0.7522297150315422


In [993]:
def standard_newton_method(f, x0, A, b, tolerance=1e-6, a=0.1, b_factor=0.5, max_iter=1000):
    grad_f = grad(f)
    hess_f = hessian(f)
    
    xt = x0
    for t in range(max_iter):
        grad_value = grad_f(xt, A, b)
        hess_value = hess_f(xt, A, b)
        
        # Add small regularization to ensure invertibility
        hess_value += 1e-5 * np.eye(hess_value.shape[0])
        
        delta_xt = np.linalg.solve(hess_value, grad_value)
        
        lambda_t = np.dot(grad_value, delta_xt)
        
        if lambda_t**2 / 2 <= tolerance and t > 0:
            break
        
        # Backtracking line search
        step_size = 1.0
        while f(xt - step_size * delta_xt, A, b) > f(xt, A, b) - a * step_size * lambda_t:
            step_size *= b_factor
        
        xt = xt - step_size * delta_xt
    
    return xt, np.abs(lambda_t)

In [1114]:
# Timing the unconstrained Newton sketch method
start_time_sketch = time.time()
optimized_x_sketch, final_lambda_sketch = unconstrained_newton_sketch(logistic_loss, x0, m, A, b, non_zero_entries)
end_time_sketch = time.time()
execution_time_sketch = end_time_sketch - start_time_sketch

# Predicting using the unconstrained Newton sketch method
y_pred_sketch = 1 / (1 + np.exp(-A @ optimized_x_sketch)) 
y_pred_sketch = (y_pred_sketch > 0.5).astype(int)

# Calculating metrics for unconstrained Newton sketch
accuracy_sketch = np.mean(y_pred_sketch == b)
precision_sketch = precision_score(b, y_pred_sketch)
recall_sketch = recall_score(b, y_pred_sketch)
f1_sketch = f1_score(b, y_pred_sketch)

# Timing the standard Newton method
start_time_standard = time.time()
optimized_x_standard, final_lambda_standard = standard_newton_method(logistic_loss, x0, A, b)
end_time_standard = time.time()
execution_time_standard = end_time_standard - start_time_standard

# Predicting using the standard Newton method
y_pred_standard = 1 / (1 + np.exp(-A @ optimized_x_standard)) 
y_pred_standard = (y_pred_standard > 0.5).astype(int)

# Calculating metrics for standard Newton method
accuracy_standard = np.mean(y_pred_standard == b)
precision_standard = precision_score(b, y_pred_standard)
recall_standard = recall_score(b, y_pred_standard)
f1_standard = f1_score(b, y_pred_standard)

# Displaying the results
print(f"Unconstrained Newton Sketch Method Results:")
print(f"Execution Time: {execution_time_sketch:.4f} seconds")
print(f"Accuracy: {accuracy_sketch * 100:.2f}%")
print(f"Precision: {precision_sketch:.4f}")
print(f"Recall: {recall_sketch:.4f}")
print(f"F1 Score: {f1_sketch:.4f}\n")

print(f"Standard Newton Method Results:")
print(f"Execution Time: {execution_time_standard:.4f} seconds")
print(f"Accuracy: {accuracy_standard * 100:.2f}%")
print(f"Precision: {precision_standard:.4f}")
print(f"Recall: {recall_standard:.4f}")
print(f"F1 Score: {f1_standard:.4f}")


0
Iteration 0 - xt update norm: 32615.42549304345
1
Iteration 1 - xt update norm: 14183.010203280794
2
Iteration 2 - xt update norm: 2446.1694235696245
3
Iteration 3 - xt update norm: 56.485094494693726
4
Iteration 4 - xt update norm: 5.254858264709294
5
Iteration 5 - xt update norm: 0.187127218652471
6
Iteration 6 - xt update norm: 0.1717349084585116
7
Iteration 7 - xt update norm: 0.15185198972110134
8
Iteration 8 - xt update norm: 0.2584985671382711
9
Iteration 9 - xt update norm: 0.15928853779582927
10
Iteration 10 - xt update norm: 0.13586895435594346
11
Iteration 11 - xt update norm: 0.14254025838733142
Unconstrained Newton Sketch Method Results:
Execution Time: 0.1163 seconds
Accuracy: 59.12%
Precision: 0.5865
Recall: 1.0000
F1 Score: 0.7394

Standard Newton Method Results:
Execution Time: 0.0568 seconds
Accuracy: 57.98%
Precision: 0.5798
Recall: 1.0000
F1 Score: 0.7340
